In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import dill as pickle

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin

class PreProcessing(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def transform(self, dataset):
        dataset = dataset.drop(['Ticket', 'Cabin'], axis=1)
        dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
        dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
        dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
        dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
        dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
        title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
        dataset['Title'] = dataset['Title'].map(title_mapping)
        dataset['Title'] = dataset['Title'].fillna(0)
        dataset = dataset.drop(['Name'], axis=1)
        dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
        dataset['Age'] = dataset['Age'].fillna(40).astype(int)
        dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
        dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
        dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
        dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
        dataset.loc[ dataset['Age'] > 64, 'Age'] = 4
        dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
        dataset['IsAlone'] = 0
        dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
        dataset = dataset.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
        dataset['Embarked'] = dataset['Embarked'].fillna("S")
        dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
        dataset['Fare'].fillna(dataset['Fare'].dropna().median(), inplace=True)
        dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
        dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
        dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
        dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
        dataset['Fare'] = dataset['Fare'].astype(int)
        return dataset.to_numpy()
    
    def fit(self, dataset, y=None):
        return self

In [4]:
pipe = Pipeline([('preprocessing', PreProcessing()), ('RF', RandomForestClassifier())])
pipe

Pipeline(steps=[('preprocessing', PreProcessing()),
                ('RF', RandomForestClassifier())])

In [5]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
X_train = df_train.drop(["Survived", "PassengerId"], axis=1)
Y_train = df_train["Survived"]
X_test  = df_test.drop("PassengerId", axis=1).copy()

In [6]:
pipe.fit(X_train, Y_train)

Pipeline(steps=[('preprocessing', PreProcessing()),
                ('RF', RandomForestClassifier())])

In [7]:
pipe.predict(X_test.loc[0:15])

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1])

In [8]:
filename = 'model_v1.pk'
with open('./'+filename, 'wb') as file:
    pickle.dump(pipe, file) 

In [9]:
with open('./'+filename ,'rb') as f:
    loaded_model = pickle.load(f)
loaded_model.predict(X_test.loc[0:15])

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1])

In [ ]:
# Reference
# https://www.kaggle.com/code/harrygem/titanic-pipeline-and-save-load-model/notebook
# https://towardsdatascience.com/creating-custom-transformers-using-scikit-learn-5f9db7d7fdb5